In [ ]:
import sys
!{sys.executable} -m pip install pymongo
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install isoweek
import os
print(os.getcwd())
# Get Mongo database
from yaml import load
from pymongo import MongoClient
from getsecret import getsecret

client = MongoClient(getsecret("MONGODB_URI"))
db = client[getsecret("DB_NAME")]
ext_client = MongoClient(getsecret("EXT_URI"))
ext_db = ext_client[getsecret("DB_NAME")]
# Get all synced accounts and their respective users.
import urllib.request as req
import json
accounts = json.loads(req.urlopen("http://localhost:5000/synced_emails").read().decode("utf-8"))
print(accounts)
# counter for figures
counter = 0
browser_freq = 0
browser_infreq = 0

In [ ]:
ANDROID = "android"
BROWSER = "browser" # habitlab goal, i.e. facebook/spend_less_time or custom/spend_less_time_developers.slashdot.org
BROWSER_DOMAIN = "browser_domain"
SHARED = "shared"
PACKAGES = "packages"
SPEND_LESS_TIME_LENGTH = len("custom/spend_less_time_")
# Associate users with domain name which will function as our key.
# Top-level-domain-names that are not pertinent to the application.
TLDs = ['aaa', 'abb', 'abc', 'ac', 'aco', 'ad', 'ads', 'ae', 'aeg', 'af', 'afl', 'ag', 'ai', 'aig', 'al', 'am', 'anz', 'ao', 'aol', 'app', 'aq', 'ar', 'art', 'as', 'at', 'au', 'aw', 'aws', 'ax', 'axa', 'az', 'ba', 'bar', 'bb', 'bbc', 'bbt', 'bcg', 'bcn', 'bd', 'be', 'bet', 'bf', 'bg', 'bh', 'bi', 'bid', 'bio', 'biz', 'bj', 'bm', 'bms', 'bmw', 'bn', 'bnl', 'bo', 'bom', 'boo', 'bot', 'box', 'br', 'bs', 'bt', 'buy', 'bv', 'bw', 'by', 'bz', 'bzh', 'ca', 'cab', 'cal', 'cam', 'car', 'cat', 'cba', 'cbn', 'cbs', 'cc', 'cd', 'ceb', 'ceo', 'cf', 'cfa', 'cfd', 'cg', 'ch', 'ci', 'ck', 'cl', 'cm', 'cn', 'co', 'com', 'cr', 'crs', 'csc', 'cu', 'cv', 'cw', 'cx', 'cy', 'cz', 'dad', 'day', 'dds', 'de', 'dev', 'dhl', 'diy', 'dj', 'dk', 'dm', 'dnp', 'do', 'dog', 'dot', 'dtv', 'dvr', 'dz', 'eat', 'ec', 'eco', 'edu', 'ee', 'eg', 'er', 'es', 'esq', 'et', 'eu', 'eus', 'fan', 'fi', 'fit', 'fj', 'fk', 'fly', 'fm', 'fo', 'foo', 'fox', 'fr', 'frl', 'ftr', 'fun', 'fyi', 'ga', 'gal', 'gap', 'gb', 'gd', 'gdn', 'ge', 'gea', 'gf', 'gg', 'gh', 'gi', 'gl', 'gle', 'gm', 'gmo', 'gmx', 'gn', 'goo', 'gop', 'got', 'gov', 'gp', 'gq', 'gr', 'gs', 'gt', 'gu', 'gw', 'gy', 'hbo', 'hiv', 'hk', 'hkt', 'hm', 'hn', 'hot', 'how', 'hr', 'ht', 'hu', 'ibm', 'ice', 'icu', 'id', 'ie', 'ifm', 'il', 'im', 'in', 'inc', 'ing', 'ink', 'int', 'io', 'iq', 'ir', 'is', 'ist', 'it', 'itv', 'jcb', 'jcp', 'je', 'jio', 'jlc', 'jll', 'jm', 'jmp', 'jnj', 'jo', 'jot', 'joy', 'jp', 'ke', 'kfh', 'kg', 'kh', 'ki', 'kia', 'kim', 'km', 'kn', 'kp', 'kpn', 'kr', 'krd', 'kw', 'ky', 'kz', 'la', 'lat', 'law', 'lb', 'lc', 'lds', 'li', 'lk', 'llc', 'lol', 'lpl', 'lr', 'ls', 'lt', 'ltd', 'lu', 'lv', 'ly', 'ma', 'man', 'map', 'mba', 'mc', 'md', 'me', 'med', 'men', 'mg', 'mh', 'mil', 'mit', 'mk', 'ml', 'mlb', 'mls', 'mm', 'mma', 'mn', 'mo', 'moe', 'moi', 'mom', 'mov', 'mp', 'mq', 'mr', 'ms', 'msd', 'mt', 'mtn', 'mtr', 'mu', 'mv', 'mw', 'mx', 'my', 'mz', 'na', 'nab', 'nba', 'nc', 'ne', 'nec', 'net', 'new', 'nf', 'nfl', 'ng', 'ngo', 'nhk', 'ni', 'nl', 'no', 'now', 'np', 'nr', 'nra', 'nrw', 'ntt', 'nu', 'nyc', 'nz', 'obi', 'off', 'om', 'one', 'ong', 'onl', 'ooo', 'org', 'ott', 'ovh', 'pa', 'pay', 'pe', 'pet', 'pf', 'pg', 'ph', 'phd', 'pid', 'pin', 'pk', 'pl', 'pm', 'pn', 'pnc', 'pr', 'pro', 'pru', 'ps', 'pt', 'pub', 'pw', 'pwc', 'py', 'qa', 'qvc', 're', 'red', 'ren', 'ril', 'rio', 'rip', 'ro', 'rs', 'ru', 'run', 'rw', 'rwe', 'sa', 'sap', 'sas', 'sb', 'sbi', 'sbs', 'sc', 'sca', 'scb', 'sd', 'se', 'ses', 'sew', 'sex', 'sfr', 'sg', 'sh', 'si', 'sj', 'sk', 'ski', 'sky', 'sl', 'sm', 'sn', 'so', 'soy', 'sr', 'srl', 'srt', 'st', 'stc', 'su', 'sv', 'sx', 'sy', 'sz', 'tab', 'tax', 'tc', 'tci', 'td', 'tdk', 'tel', 'tf', 'tg', 'th', 'thd', 'tj', 'tjx', 'tk', 'tl', 'tm', 'tn', 'to', 'top', 'tr', 'trv', 'tt', 'tui', 'tv', 'tvs', 'tw', 'tz', 'ua', 'ubs', 'ug', 'uk', 'uno', 'uol', 'ups', 'us', 'uy', 'uz', 'va', 'vc', 've', 'vet', 'vg', 'vi', 'vig', 'vin', 'vip', 'vn', 'vu', 'wed', 'wf', 'win', 'wme', 'wow', 'ws', 'wtc', 'wtf', 'xin', 'xxx', 'xyz', 'ye', 'you', 'yt', 'yun', 'za', 'zip', 'zm', 'zw']
def get_name(name, device):
    """
    @param name: goal name (package name for Android)
    @param device: "android" or "browser" or "browser_domain"
    @return name of goal with subdomains removed and goal annotation removed (i.e. spend_less_time)
    """
    name = name.lower()
    if "custom" in name and device == BROWSER:
        # strip off the "custom/spend_less_time_"
        name = name[SPEND_LESS_TIME_LENGTH:]
    elif device == BROWSER:
        return name.split('/spend')[0]
    # Now we have to get juicy part of domain.
    subs = list(filter(lambda x: x != "android" and x != "google" and x != "apps" and x not in TLDs, name.split('.')))
    if device == ANDROID: 
        if len(subs) > 0:
            return subs[0]
        return name
    else:
        if len(subs) > 0:
            return subs[len(subs) - 1]    
        
def organize_stats(shared_goals, stats, device, counts, user_id):
    """
    Organizes that stats object into shared_goals for device.
    @param shared_goals: dictionary
    @param stats:  stats object returned from freq_stats
    @param device: ANDROID or BROWSER
    """
    for iso in stats:
        for freq in stats[iso]:
            for goal in stats[iso][freq]:
                name = get_name(goal, device)
                if name not in shared_goals:
                    shared_goals[name] = {ANDROID: {PACKAGES:[]} , BROWSER: {PACKAGES:[]} }
                if goal not in shared_goals[name][device][PACKAGES]:
                    shared_goals[name][device][PACKAGES].append(goal)
                    shared_goals[name][device][goal] = {}
                if iso not in shared_goals[name][device][goal]:
                    shared_goals[name][device][goal][iso] = freq
                    if device == BROWSER:
                        counts[freq] += 1
                # Before I submitted the update, some apps under the same name wouldn't have the same freq setting.
                elif shared_goals[name][device][goal][iso] != freq:
                    print(shared_goals[name][device][goal])
                    print(user_id + " " + device + " " + goal + " " + iso + " " + shared_goals[name][device][goal][iso] + " " + freq)
                    shared_goals[name][device][goal][iso] = "both"
                    counts["both"] += 1
                if len(shared_goals[name][ANDROID][PACKAGES]) > 0 and len(shared_goals[name][BROWSER][PACKAGES]) > 0:
                    shared_goals[SHARED].add(name)



In [ ]:
# Analyze conservation across devices
shared_data = {}
counts = {"freq" : 0, "infreq": 0,  'both' :0}

num_shared_goals = 0
for account in accounts:
    shared_goals = {SHARED: set([])}
    if len(account[ANDROID]) > 0 and len(account[BROWSER]) > 0: # It's possible they have overlapping goals.
        if len(account[BROWSER]) > 0:
            print("BROWSER USERS for " + str(account))
        for user in account[ANDROID]:
            stats = json.loads(req.urlopen("http://localhost:5000/freq_stats_for_user?id="+user).read().decode("utf-8"))
            organize_stats(shared_goals, stats, ANDROID, counts, user)
        for user in account[BROWSER]:
            stats = json.loads(req.urlopen('http://localhost:5000/freq_stats_for_user_browser?id='+user).read().decode("utf-8"))
            organize_stats(shared_goals, stats, BROWSER, counts, user)
        if len(shared_goals[SHARED]) > 0:
            shared_data[account['_id']] = shared_goals 
            num_shared_goals += len(shared_goals[SHARED])
print(str(num_shared_goals) + " shared goals")

In [ ]:
print(str(counts))
print(shared_data)

In [ ]:
# count how many freq vs. in freq things there are in shared_data
device_freq = {"android": {"freq" : 0, "infreq": 0, "both": 0},  "browser": {"freq":  0, "infreq": 0, "both": 0}}
for account in shared_data:
    for goal in shared_data[account]:
        if goal != "shared": 
            for device in shared_data[account][goal]:
                for package in shared_data[account][goal][device]:
                    if package != "packages":
                        for week in shared_data[account][goal][device][package]:
                            print(goal)
                            device_freq[device][shared_data[account][goal][device][package][week]] += 1
print(device_freq)
#Hmmmm... android looks okay, but big imbalance between freq/infreq browser

In [ ]:
# Let's see what browser names there are
browser_names = set([])
for account in shared_data:
    for goal in shared_data[account]:
        if BROWSER in shared_data[account][goal]:
            if len(shared_data[account][goal][BROWSER]["packages"]) > 1:
                browser_names.add(goal + " " + str(shared_data[account][goal][BROWSER]))

In [ ]:
print(browser_names)

In [ ]:
def get_iso_for_week(iso_week, account, shared_data, goal):
    """
    Gets browser and android frequency settings for a given goal, week, and account.
    :account email hash
    :goal get_name return of package
    :shared_data {email_hash: 'shared': {set of shared goals}, goal: {device: {'packages': [], package: {isoWeek: 'freq' or 'infreq'}}}}
    :return (browser frequency -> "freq" or "infreq", android frequency -> "freq" or "infreq"); if no freq found None
    """
    # First, get browser package
    browser_goal_packages = shared_data[account][goal][BROWSER][PACKAGES]
    browser_freq = None
    for package in browser_goal_packages:
        if not browser_freq:
            for iso in shared_data[account][goal][BROWSER][package]:
                if str(iso) == str(iso_week):
                    browser_freq = shared_data[account][goal][BROWSER][package][iso]
                    break
    if not browser_freq:
        return None
    # Next get Android packages
    android_goal_packages = shared_data[account][goal][ANDROID][PACKAGES]
    android_freq = None
    for package in android_goal_packages:
        if not android_freq:
            for iso in shared_data[account][goal][ANDROID][package]:
                if str(iso) == str(iso_week):
                    android_freq = shared_data[account][goal][ANDROID][package][iso]
                    break
    if not android_freq:
        return None
    return (browser_freq, android_freq)


In [ ]:
from isoweek import Week
from datetime import date, datetime
import moment
import math
from time_utils import epoch_to_date
# Now, the goal is to get the daily time spent on these special domains.
# Goal: [{"goal": "", "device": "android", frequent": true, "email_hash": "", "other_device_freq": true}]
OTHER_FREQUENCY = "other_frequency"
TIME = "time"
device_freq = {"android": {"freq" : 0, "infreq": 0, "both": 0},  "browser": {"freq":  0, "infreq": 0, "both": 0}}
day_spent_shared_domain_stats = []
for account in shared_data:
    print(account)
    # The browser data is already there, so let's fetch it.
    domain_freq_day_time = {BROWSER: {}, ANDROID: {}}
    for user in db['email_to_user'].find({'_id': account})[0][BROWSER]:
        for day_stat in ext_db[user+ "_synced:seconds_on_domain_per_day"].find():
            goal = get_name(day_stat['key'], BROWSER_DOMAIN)
            # print(shared_data[account][SHARED])
            if goal in shared_data[account][SHARED]: # It can be a data point.
                # The key1 serves as a day. However, if we are going to compare same day stuffs for android.
                # We need to instead build day string from moment
                print(str(day_stat))
                day = epoch_to_date(int(day_stat['key2']))
                # weirdly enough, datetime timestamps are seconds, not ms.
                isoWeek = str(datetime.fromtimestamp(moment.date(day).epoch()).isocalendar()[1])
                # Now, we need to hope that all the packages the user is using under that goal name is in 
                # same freq/infreq setting.
                freq_settings = get_iso_for_week(isoWeek, account, shared_data, goal)
                if freq_settings:
                    (browser_freq, android_freq) = freq_settings
                    # YES WE HAVE ALL THE DATAS FOR A POINT
                    if goal not in domain_freq_day_time[BROWSER]:
                        domain_freq_day_time[BROWSER][goal] = {}
                    if browser_freq not in domain_freq_day_time[BROWSER][goal]:
                        domain_freq_day_time[BROWSER][goal][browser_freq] = {}
                    if day not in domain_freq_day_time[BROWSER][goal][browser_freq]:
                        domain_freq_day_time[BROWSER][goal][browser_freq][day] = {}
                    day_obj = domain_freq_day_time[BROWSER][goal][browser_freq][day]
                    if OTHER_FREQUENCY not in day_obj:
                        day_obj[OTHER_FREQUENCY] = android_freq
                    if TIME not in day_obj or math.log(day_stat['val']) > day_obj[TIME]:
                        # we want the final update of time spent for that day on that domain.
                        day_obj[TIME] = (math.log(day_stat['val']))
    """
    for user in db['email_to_user'].find({'_id': account})[0][ANDROID]:
        for s in db[user + '_sessions'].find():
            goal = get_name(s['domain'], ANDROID)
            if goal in shared_data[account][SHARED]:
                time = moment.unix(day_stat['timestamp'])
                day = time.format('YYYY-MM-DD')
                isoWeek = str(datetime.fromtimestamp(s["timestamp"]/1000).isocalendar()[1])
                # Now, we need to hope that all the packages the user is using under that goal name is in 
                # same freq/infreq setting.
                freq_settings = get_iso_for_week(isoWeek, account, shared_data, goal)
                if freq_settings:
                    (browser_freq, android_freq) = freq_settings
                    # YES WE HAVE ALL THE DATAS FOR A POINT
                    if goal not in domain_freq_day_time[ANDROID]:
                        domain_freq_day_time[ANDROID][goal] = {}
                    if android_freq not in domain_freq_day_time[ANDROID][goal]:
                        domain_freq_day_time[ANDROID][goal][android_freq] = {}
                    if day not in domain_freq_day_time[ANDROID][goal][android_freq]:
                        domain_freq_day_time[ANDROID][goal][android_freq][day] = {}
                    day_obj = domain_freq_day_time[ANDROID][goal][android_freq][day]
                    if OTHER_FREQUENCY not in day_obj:
                        day_obj[OTHER_FREQUENCY] = browser_freq
                    if s['duration'] > 0 and (TIME not in day_obj or math.log(s['duration']) > day_obj[TIME]):
                        # we want the final update of time spent for that day on that domain.
                        day_obj[TIME] = math.log(s['duration'])
            This script uses domain_stats. Not of interest yet.
            for day in goal_obj:
                if day != '_id':
                    # Moment needs dashes
                    day_time = day[:4] + '-' + day[4:6] + '-' + day[-2:]
                    # Now, let's get the iso week
                    week = datetime.fromtimestamp(moment.date(day_time).epoch()).isocalendar()[1]
                    # same freq/infreq setting.
                    freq_settings = get_iso_for_week(isoWeek, account, shared_data, goal)
                    if freq_settings:
                        (browser_freq, android_freq) = freq_settings
                        # YES WE HAVE ALL THE DATAS FOR A POINT
                        if goal not in domain_freq_day_time[ANDROID]:
                            domain_freq_day_time[ANDROID][goal] = {}
                        if browser_freq not in domain_freq_day_time[ANDROID][goal]:
                            domain_freq_day_time[ANDROID][goal][android] = {}
                        if day not in domain_freq_day_time[ANDROID][goal][android_freq]:
                            domain_freq_day_time[ANDROID][goal][browser_freq][day] = {}
                        day_obj = domain_freq_day_time[ANDROID][goal][android_freq][day]
                        if OTHER_FREQUENCY not in day_obj:
                            day_obj[OTHER_FREQUENCY] = browser_freq
                        if TIME not in day_obj or goal_obj[day] > day_obj[TIME]:
                            # we want the final update of time spent for that day on that domain.
                            day_obj[TIME] = math.log(goal_obj[day])
    """

    # PHEW WE GOT THRU IT ALL. Now let's add the valid data points to the array!

    for device in domain_freq_day_time:
        for goal in domain_freq_day_time[device]:
            for freq in domain_freq_day_time[device][goal]:
                for day in domain_freq_day_time[device][goal][freq]:
                    day_obj = domain_freq_day_time[device][goal][freq][day]
                    device_freq[device][freq] += 1
                    obj = {"goal": goal, "device": device, "frequent": freq, "email_hash": account,
                          "other_device_freq": day_obj[OTHER_FREQUENCY], "duration":day_obj[TIME], "day": day}
                    day_spent_shared_domain_stats.append(obj)

In [ ]:
# Now do this for Android!
for account in shared_data:
    # The browser data is already there, so let's fetch it.
    domain_freq_day_time = {BROWSER: {}, ANDROID: {}}
    for user in db['email_to_user'].find({'_id': account})[0][ANDROID]: 
        for s in db[user + '_sessions'].find():
            goal = get_name(s['domain'], ANDROID)
            if goal in shared_data[account][SHARED]: 
                # The key1 serves as a day. However, if we are going to compare same day stuffs for android.
                # We need to instead build day string from moment
                time = moment.unix(s["timestamp"])
                day = time.format("YYYY-MM-DD")
                # weirdly enough, datetime timestamps are seconds, not ms.
                isoWeek = str(datetime.fromtimestamp(day_stat["timestamp"]/1000).isocalendar()[1]) 
                # Now, we need to hope that all the packages the user is using under that goal name is in 
                # same freq/infreq setting.
                freq_settings = get_iso_for_week(isoWeek, account, shared_data, goal)
                if freq_settings:
                    (browser_freq, android_freq) = freq_settings
                    # YES WE HAVE ALL THE DATAS FOR A POINT
                    if goal not in domain_freq_day_time[ANDROID]:
                        domain_freq_day_time[ANDROID][goal] = {}
                    if browser_freq not in domain_freq_day_time[ANDROID][goal]:
                        domain_freq_day_time[ANDROID][goal][android_freq] = {}
                    if day not in domain_freq_day_time[ANDROID][goal][android_freq]:
                        domain_freq_day_time[ANDROID][goal][android_freq][day] = {}
                    day_obj = domain_freq_day_time[ANDROID][goal][android_freq][day]
                    if s['duration'] > 0 and (TIME not in day_obj or math.log(s['duration']) > day_obj[TIME]):
                        # we want the final update of time spent for that day on that domain.
                        day_obj[TIME] = (math.log(s['duration']))
                    if TIME in day_obj and OTHER_FREQUENCY not in day_obj:
                        day_obj[OTHER_FREQUENCY] = browser_freq
                    

    # PHEW WE GOT THRU IT ALL. Now let's add the valid data points to the array!
    for device in domain_freq_day_time:
        for goal in domain_freq_day_time[device]:
            for freq in domain_freq_day_time[device][goal]:
                for day in domain_freq_day_time[device][goal][freq]:
                    day_obj = domain_freq_day_time[device][goal][freq][day]
                    device_freq[device][freq] += 1
                    print(day_obj)
                    if OTHER_FREQUENCY in day_obj and "duration" in day_obj:
                        obj = {"goal": goal, "device": device, "frequent": freq, "email_hash": account,
                              "other_device_freq": day_obj[OTHER_FREQUENCY], "duration":day_obj[TIME], "day": day}
                        day_spent_shared_domain_stats.append(obj)

In [ ]:
print(len(day_spent_shared_domain_stats))
print(day_spent_shared_domain_stats)
print(device_freq)

In [ ]:
# enables the %%R magic, not necessary if you've already done this
%load_ext rpy2.ipython
import pandas as pd
import json

In [ ]:
# Put this into data frame
# precondition: day_spent_shared_domain_stats has at least one data point.
data_frame_dict = { key: [] for key in day_spent_shared_domain_stats[0] }
# Let's also count up the freq to see if they're roughly equal
field_stats = {}
for item in day_spent_shared_domain_stats:
    if item['frequent'] != 'both' and item["other_device_freq"] != "both":
        if item['device'] not in field_stats:
            field_stats[item['device']] = {}
        if item["frequent"] not in field_stats[item['device']]:
            field_stats[item['device']][item["frequent"]] = 1
        else:
            field_stats[item['device']][item["frequent"]] += 1
        for key in item:
            data_frame_dict[key].append(item[key])
df = pd.DataFrame(data_frame_dict)


In [ ]:
import matplotlib.pyplot as pyplot
import math
counter += 1
for item in day_spent_shared_domain_stats:
    if item['device'] == ANDROID and item['frequent'] == "freq":
        print((item['duration']))
pyplot.figure(counter)
durations = [(day['duration']) for day in day_spent_shared_domain_stats if math.exp(day['duration']) < 100 and day['device'] == ANDROID and day['frequent'] == 'freq']
pyplot.hist(durations, bins=200)
import statistics
print(statistics.mean(durations))

In [ ]:
%%R -i df -w 5 -h 5 --units in -r 200

#install.packages('ez')
#install.packages('lme4')

library(lme4)
library(sjPlot)
library(lmerTest)
print(df)
#library(ez)

In [ ]:
%%R -i df -w 5 -h 5 --units in -r 200


# SIGNIFCANT: does the frequency have an effect on duration, controlling for user and goal?
df$frequent <- factor(df$frequent, levels=c("infreq", "freq"), ordered=FALSE)
df$device <- factor(df$device, levels=c("browser", "android"), ordered=FALSE)
df$other_device_freq <- factor(df$other_device_freq, levels=c("infreq", "freq"), ordered=FALSE)
df$email_hash <- factor(df$email_hash,  ordered=FALSE)

results <- lmer(duration ~ (other_device_freq) + (frequent) + (1|email_hash) + (1|goal) + device, data = df, levels=c("infreq","freq"))

resultsnull <- lmer(duration ~  (1|email_hash) + (1|goal), data = df)

show(results)
sjp.lmer(results)
show(ranef(results))
show(fixef(results))
summary(results)

In [ ]:
print(str(field_stats))

In [ ]:
# Next, let's implement Michael's test.
# First, we need aggregate time spent across devices for a certain goal.
BROWSER_FREQ = "browser_freq"
ANDROID_FREQ = "android_freq"
agg_stat = {}
for item in day_spent_shared_domain_stats:
    email = item["email_hash"]
    if email not in agg_stat:
        agg_stat[email] = {}
    date = item["day"]
    goal = item["goal"]
    if goal not in agg_stat[email]:
        agg_stat[email][goal] = {}
    if date not in agg_stat[email][goal]:
        agg_stat[email][goal][date] = {}
    if BROWSER_FREQ not in agg_stat[email][goal][date]:
        if item["device"] == BROWSER:
            agg_stat[email][goal][date][BROWSER_FREQ] = item["frequent"]
            agg_stat[email][goal][date][ANDROID_FREQ] = item["other_device_freq"]
        else:
            agg_stat[email][goal][date][BROWSER_FREQ] = item["other_device_freq"]
            agg_stat[email][goal][date][ANDROID_FREQ] = item["frequent"]
    if "duration" not in agg_stat[email][goal][date]:
        agg_stat[email][goal][date]["duration"] = item["duration"]
    else:
        agg_stat[email][goal][date]["duration"] += item["duration"]

In [ ]:
agg_stat_list = []
for email in agg_stat:
    for goal in agg_stat[email]:        
        for date in agg_stat[email][goal]:
            obj = agg_stat[email][goal][date]
            agg_stat_list.append({"duration": obj["duration"], BROWSER_FREQ: obj[BROWSER_FREQ], ANDROID_FREQ: obj[ANDROID_FREQ],
                                 "email_hash": email, "goal": goal})

In [ ]:
print(agg_stat_list)

In [ ]:
"""
Linear mixed model fit by REML ['lmerMod']
Formula: duration ~ other_device_freq + (1 | frequent) + (1 | email_hash) +  
    (1 | goal) + (1 | device)
   Data: df
REML criterion at convergence: 1219.07
Random effects:
 Groups     Name        Std.Dev.
 email_hash (Intercept) 1.1805  
 goal       (Intercept) 0.0000  
 device     (Intercept) 0.6750  
 frequent   (Intercept) 0.2812  
 Residual               1.5227  
Number of obs: 316, groups:  email_hash, 29; goal, 9; device, 2; frequent, 2
Fixed Effects:
            (Intercept)  other_device_freqinfreq  
                 5.1227                   0.7823  
Data: df
Models:
resultsnull: duration ~ (1 | frequent) + (1 | email_hash) + (1 | goal) + (1 | 
resultsnull:     device)
results: duration ~ other_device_freq + (1 | frequent) + (1 | email_hash) + 
results:     (1 | goal) + (1 | device)
            Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
resultsnull  6 1241.6 1264.1 -614.78   1229.6                             
results      7 1232.4 1258.7 -609.21   1218.4 11.149      1  0.0008407 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
"""

In [ ]:
data_frame_dict = { key: [] for key in agg_stat_list[0] }
for item in agg_stat_list:
    for key in item:
        data_frame_dict[key].append(item[key])

df = pd.DataFrame(data_frame_dict)


In [ ]:
%%R -i df -w 5 -h 5 --units in -r 200

# SIGNIFCANT: does the frequency have an effect on duration, controlling for user and goal?

results <- lmer(duration ~ (android_freq) + (browser_freq) + (1|email_hash) + (1|goal) , data = df)

resultsnull <- lmer(duration ~  (1|email_hash) + (1|goal), data = df)

show(results)
show(anova(resultsnull, results))

In [ ]:
"""
# Time on both

time ~ mobilecond desktopcond mobile * desktop
do two separate tests
"""
